In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
import pickle

In [2]:
fulldata = pd.read_pickle('full_metadata.pkl')

In [3]:
fulldata.shape

(51487, 15)

In [4]:
fulldata.head(2)

,isbn,user_id,user_name,user_ratings,user_review_text,authors,categories,description,language,pageCount,printType,publishedYear,publisher,title,weighted_avg
0,9780321948540,779608,Erin,4.0,"There are a lot of great tips in this volume, ...",Scott Kelby,Computers,"Furnishes an overview of digital photography, ...",en,236.0,BOOK,2013.0,Pearson Education,The Digital Photography Book,4.128036
1,9780321948540,753824,Rolf Häsänen,4.0,Not as many useful tips as volume1 unless you ...,Scott Kelby,Computers,"Furnishes an overview of digital photography, ...",en,236.0,BOOK,2013.0,Pearson Education,The Digital Photography Book,4.128036


In [5]:
#Get list of all isbns
allisbns = fulldata[['isbn']]

In [6]:
allisbns.isbn.unique()

array(['9780321948540', '9780133856934', '9780133856880', ...,
       '9781591582830', '9781551801049', '9781593500146'], dtype=object)

In [7]:
allisbns.reset_index(inplace=True)

In [8]:
allisbns.drop(columns=['index'],inplace=True)

/Users/sameerdeekshit/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
allisbns.drop_duplicates(inplace=True)
allisbns.head()

/Users/sameerdeekshit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,isbn
0,9780321948540
30,9780133856934
39,9780133856880
57,9781429957113
87,9780134385273


In [10]:
all_isbn_list = allisbns['isbn'].tolist()

In [11]:
allisbnlistfile = open("all_isbn_list.pkl","wb")
pickle.dump(all_isbn_list,allisbnlistfile)
allisbnlistfile.close()

In [12]:
#Get list of all isbns and book titles
isbn_title_df = fulldata[['isbn','title']]

In [13]:
isbn_title_df.shape

(51487, 2)

In [14]:
isbn_title_df.drop_duplicates(inplace=True)

/Users/sameerdeekshit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
isbn_title_df.reset_index(inplace=True)

In [16]:
isbn_title_df.drop_duplicates(inplace=True)
isbn_title_df.head()

/Users/sameerdeekshit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,index,isbn,title
0,0,9780321948540,The Digital Photography Book
1,30,9780133856934,"The Digital Photography Book, Part 5"
2,60,9780133856880,The Digital Photography Book
3,90,9781429957113,On Photography
4,169,9780134385273,The Best of The Digital Photography Book Series


In [17]:
isbn_title_df.drop(columns=['index'],inplace=True)

In [18]:
isbn_title_df.head()

,isbn,title
0,9780321948540,The Digital Photography Book
1,9780133856934,"The Digital Photography Book, Part 5"
2,9780133856880,The Digital Photography Book
3,9781429957113,On Photography
4,9780134385273,The Best of The Digital Photography Book Series


In [19]:
isbn_title_df = isbn_title_df.loc[isbn_title_df['isbn'].isin(all_isbn_list)]

In [20]:
isbn_title_df.head()

,isbn,title
0,9780321948540,The Digital Photography Book
1,9780133856934,"The Digital Photography Book, Part 5"
2,9780133856880,The Digital Photography Book
3,9781429957113,On Photography
4,9780134385273,The Best of The Digital Photography Book Series


In [21]:
isbn_title_dict = pd.Series(isbn_title_df.title.values,index=isbn_title_df.isbn).to_dict()

In [22]:
isbn_title_dict

{'9780321948540': 'The Digital Photography Book',
 '9780133856934': 'The Digital Photography Book, Part 5',
 '9780133856880': 'The Digital Photography Book',
 '9781429957113': 'On Photography',
 '9780134385273': 'The Best of The Digital Photography Book Series',
 '9780133510720': 'The Digital Photography Book',
 '9780415307031': 'Photography',
 '9781579907594': 'The Complete Guide to Digital Photography',
 '9780080476926': 'Crime Scene Photography',
 '9780321686008': "Scott Kelby's Digital Photography Books, Volumes 1, 2, and 3",
 '9780393050240': 'After Photography',
 '9781780235615': 'On Photography',
 '9781579905279': 'Portrait Photography',
 '9781579907723': 'Complete Guide to Digital Infrared Photography',
 '9781856694933': 'Photography',
 '9781579907624': 'The Kodak Most Basic Book of Digital Photography',
 '9781845112233': 'Street Photography',
 '9780811733496': 'Fine Art Photography',
 '9780374521349': 'Camera Lucida',
 '9780486251288': 'A Concise History of Photography',
 '978

In [23]:
isbntitlefile = open("isbn_title_dict.pkl","wb")
pickle.dump(isbn_title_dict,isbntitlefile)
isbntitlefile.close()

In [24]:
isbn_title_dict_new = pickle.load(open( "isbn_title_dict.pkl", "rb" ) )

In [25]:
reviews = pd.read_pickle('processed_reviews.pkl')

In [26]:
reviews.shape

(48201, 5)

In [27]:
reviews.head()

,isbn,user_id,user_name,user_ratings,user_review_text
0,9780321948540,779608,Erin,4.0,"There are a lot of great tips in this volume, ..."
1,9780321948540,753824,Rolf Häsänen,4.0,Not as many useful tips as volume1 unless you ...
2,9780321948540,35798331,Valery,4.0,"Not as helpful as the first, but does go into ..."
3,9780321948540,2100772,Bruce,1.0,"OMG. Surely the absolute worst, amature photo ..."
4,9780321948540,1794100,Icepick,5.0,"The format may not be for everybody, but I rea..."


In [28]:
reviews.isnull().sum()

isbn                0
user_id             0
user_name           0
user_ratings        0
user_review_text    0
dtype: int64

In [29]:
isbn_users_rating = reviews[['user_id', 'isbn', 'user_ratings']]
isbn_users_rating.head()

,user_id,isbn,user_ratings
0,779608,9780321948540,4.0
1,753824,9780321948540,4.0
2,35798331,9780321948540,4.0
3,2100772,9780321948540,1.0
4,1794100,9780321948540,5.0


In [30]:
isbn_users_rating.shape

(48201, 3)

In [31]:
isbn_users_rating = isbn_users_rating.loc[isbn_users_rating['isbn'].isin(all_isbn_list)]

In [32]:
isbn_users_rating.shape

(43028, 3)

In [33]:
isbn_users_rating.head()

,user_id,isbn,user_ratings
0,779608,9780321948540,4.0
1,753824,9780321948540,4.0
2,35798331,9780321948540,4.0
3,2100772,9780321948540,1.0
4,1794100,9780321948540,5.0


In [34]:
#Display top users group by userid
isbn_users_rating.groupby('user_id')['isbn'].nunique().sort_values(ascending=False)

user_id
4213258     63
4622890     51
597461      50
5253785     48
269235      45
45618       39
405390      38
614778      37
416390      36
17438949    36
1413439     35
3978225     34
68030       34
4159922     34
1713956     33
16254355    32
16731747    31
48483884    30
279256      30
1742824     29
3452652     29
2457095     28
5903843     26
8734459     26
8655084     26
14596170    25
10171516    25
1237196     25
2294090     24
3427339     24
            ..
50430625     1
50432252     1
5043487      1
50436569     1
50437033     1
5038871      1
5038841      1
50388253     1
5033113      1
5028071      1
5028119      1
5028755      1
5029381      1
5030336      1
50304360     1
5032112      1
5032407      1
5032595      1
5033175      1
50387        1
5033258      1
5033403      1
5033508      1
5034368      1
5034547      1
50363505     1
503678       1
5037696      1
5037956      1
3718814      1
Name: isbn, Length: 27587, dtype: int64

In [35]:
isbn_users_rating.isnull().sum()

user_id         0
isbn            0
user_ratings    0
dtype: int64

In [36]:
#Generate ratings matrix
ratings_matrix = isbn_users_rating.pivot_table(index='user_id', columns='isbn', values='user_ratings')
userid = ratings_matrix.index
isbn = ratings_matrix.columns

In [37]:
print(ratings_matrix.shape)
ratings_matrix.head()

(27587, 2279)


isbn,9780007198238,9780007236350,9780007340484,9780007509263,9780008291730,9780028612775,9780030529474,9780060628727,9780060754730,9780060821982,...,9788496517103,9788590379829,9789004106673,9789042019485,9789042911833,9789380028408,9789384049263,9789386224897,9789386867261,9789812382924
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100026730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
#Fill na
ratings_matrix.fillna(0, inplace = True)
ratings_matrix = ratings_matrix.astype(np.int32)

In [39]:
ratings_matrix.head(5)

isbn,9780007198238,9780007236350,9780007340484,9780007509263,9780008291730,9780028612775,9780030529474,9780060628727,9780060754730,9780060821982,...,9788496517103,9788590379829,9789004106673,9789042019485,9789042911833,9789380028408,9789384049263,9789386224897,9789386867261,9789812382924
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10001960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100026730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
ratings_matrix.to_pickle('collaborative_ratings_matrix.pkl')

In [41]:
#Calculate sparsity of ratings matrix
sparsity=1.0-len(ratings_matrix)/float(ratings_matrix.shape[0]*ratings_matrix.shape[1])
print ("sparsity percentage : " +  str(sparsity*100))

sparsity percentage : 99.95612110574812


In [42]:
#Calculate pairwise distance similarity
def closest_users(user, ratings, n=10):
    model = NearestNeighbors(metric = 'cosine', algorithm = 'brute') 
    model.fit(ratings)
    index = ratings.index.get_loc(user)
    distances, indices = model.kneighbors(ratings.iloc[index, :].values.reshape(1, -1), n_neighbors = n+1)
    distances = 1-distances.flatten()
    return distances,indices

In [43]:
#User based similarity
def predict_ratings_user(user, item, ratings, n=10):
    total=0
    distances, indices= closest_users(user, ratings, n)
    rmean = ratings.iloc[ratings.index.get_loc(user),:].mean()
    for x in range(0, len(indices.flatten())):
        if indices.flatten()[x] != ratings.columns.get_loc(item):
            diff = ratings.iloc[indices.flatten()[x],ratings.columns.get_loc(item)]-np.mean(ratings.iloc[indices.flatten()[x],:])
            delta = diff * (distances[x])
            total = total + delta
    return int(round(rmean + (total/(np.sum(distances)-1))))

In [44]:
testr = predict_ratings_user('779608','9780321948540',ratings_matrix);
print(testr)

4


In [45]:
testr  = predict_ratings_user('27610501','9780133856934',ratings_matrix);
print(testr)

5


In [46]:
#Item based
def closest_items(item, ratings, n=10):
    ratings=ratings.T
    index = ratings.index.get_loc(item)
    model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
    model.fit(ratings)
    distances, indices = model.kneighbors(ratings.iloc[index, :].values.reshape(1, -1), n_neighbors = n+1)
    distances = 1-distances.flatten()
    return distances,indices

In [47]:
#predict ratings
def predict_ratings_item(user, item, ratings, n=10):
    total = 0
    distances, indices=closest_items(item, ratings)
    for x in range(0, len(indices.flatten())):
        if indices.flatten()[x] != ratings.columns.get_loc(item):
            factor = ratings.iloc[ratings.index.get_loc(user),indices.flatten()[x]] * (distances[x])
            total = total + factor
            
    try :
        item_rating = int(round(total/(np.sum(distances)-1)))
    except:
        item_rating=0
    
    return item_rating

In [48]:
prediction = predict_ratings_item('779608','9780321948540',ratings_matrix)
print(prediction)

2


In [49]:
#get isbn from recommended ratings
def top10isbn(recommendations):
    top10_isbn = []
    for item in recommendations:
        item_isbn = item[1]
        top10_isbn.append(item_isbn)
    return top10_isbn

In [50]:
#Get top user baed collborative filtering recommendations for user
def ubcftop10(user):
    ratings_rankings = []
    for i in range(ratings_matrix.shape[1]):
        ratinglist = []
        if ratings_matrix[str(ratings_matrix.columns[i])][user] ==0:
            ratinglist.append(int(-1))
            ratinglist.append(str(ratings_matrix.columns[i]))
        else:    
            ratinglist.append(int((predict_ratings_user(user, str(ratings_matrix.columns[i]),ratings_matrix)))) 
            ratinglist.append(str(ratings_matrix.columns[i]))
        ratings_rankings.append(ratinglist)
    
    ratings_rankings = [item for item in ratings_rankings if item[1] in all_isbn_list]
    ratings_rankings.sort(key=lambda x: x[0])
    ratings_rankings.reverse()
    top10_ratings = ratings_rankings[:10]
    return top10_ratings   

In [51]:
recommendations = ubcftop10('269235')
isbn_recommendations = top10isbn(recommendations)
print("User based collaborative filtering recommendations : ")
for isbnitem in isbn_recommendations :
    print("ISBN :", str(isbnitem), "Title :", str(isbn_title_dict[isbnitem]))

User based collaborative filtering recommendations : 
ISBN : 9781613126219 Title : El Deafo
ISBN : 9781770461987 Title : SuperMutant Magic Academy
ISBN : 9781770461536 Title : Over Easy
ISBN : 9781596436626 Title : Red Handed
ISBN : 9781466858527 Title : This One Summer
ISBN : 9781442465978 Title : Through the Woods
ISBN : 9780679462712 Title : Into Thin Air
ISBN : 9780618871711 Title : Fun Home
ISBN : 9780307772947 Title : The Postman Always Rings Twice
ISBN : 9780307767516 Title : The Maltese Falcon


In [52]:
#Get top item baed collborative filtering recommendations for user
def ibcftop10(user):
    ratings_rankings = []
    for i in range(ratings_matrix.shape[1]):
        ratinglist = []
        if ratings_matrix[str(ratings_matrix.columns[i])][user] ==0:
            ratinglist.append(int(-1))
            ratinglist.append(str(ratings_matrix.columns[i]))
        else:    
            ratinglist.append(int((predict_ratings_item(user, str(ratings_matrix.columns[i]),ratings_matrix)))) 
            ratinglist.append(str(ratings_matrix.columns[i]))
        ratings_rankings.append(ratinglist)
    
    ratings_rankings = [item for item in ratings_rankings if item[1] in all_isbn_list]
    ratings_rankings.sort(key=lambda x: x[0])
    ratings_rankings.reverse()
    top10_ratings = ratings_rankings[:10]
    return top10_ratings   

In [53]:
recommendations = ibcftop10('269235')
isbn_recommendations = top10isbn(recommendations)
print("Item based collaborative filtering recommendations : ")
for isbnitem in isbn_recommendations :
    print("ISBN :", str(isbnitem), "Title :", str(isbn_title_dict[isbnitem]))

Item based collaborative filtering recommendations : 
ISBN : 9781939799081 Title : Infomaniacs
ISBN : 9781910593554 Title : I Feel Machine
ISBN : 9781770461536 Title : Over Easy
ISBN : 9781596438736 Title : Hidden: A Child's Story of the Holocaust
ISBN : 9780224063975 Title : Jimmy Corrigan
ISBN : 9781770463219 Title : A Bubble
ISBN : 9781770463165 Title : Sabrina
ISBN : 9781770461987 Title : SuperMutant Magic Academy
ISBN : 9781683961895 Title : Alienation
ISBN : 9781613126219 Title : El Deafo
